In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
from pandas_plink import read_plink
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from joblib import Parallel, delayed
from sklearn import metrics
from math import sqrt

import random
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import log_loss

In [ ]:
''' Parameters for experiment '''
threshold="0.001"
path_logs="/work/breastcancer/clean_test/logs/"
path_to_files="/work/breastcancer/clean_test/"
''' getting bim,fam,bed for training,validation and test sets '''
(bim, fam, bed)=read_plink(path_to_files+"train/sig"+threshold)
(bim2, fam2, bed2)=read_plink(path_to_files+"validation/val"+threshold)
(bim3, fam3, bed3)=read_plink(path_to_files+"test/test"+threshold)

path_logs="/work/breastcancer/clean_test/logs/"

print(bim)

print(fam)

''' Creating arrays with optimal data structure and filling missing values with 2--> Homozygous major '''
print("Convertion")
bed=bed.astype('uint8')
print("Compute")
X=bed.compute()
print("Filling Null Data")
X[np.isnan(X)]=2
#validation
print("Convertion")
bed2=bed2.astype('uint8')
print("Compute")
X_val=bed2.compute()
print("Filling Null Data")
X_val[np.isnan(X_val)]=2
#test
print("Convertion")
bed3=bed3.astype('uint8')
print("Compute")
X_test=bed3.compute()
print("Filling Null Data")
X_test[np.isnan(X_test)]=2

''' Preparing data.shape=(individuals,SNP) '''
#train
Y=fam["trait"].astype("int")
Y=Y-1
Xdf=pd.DataFrame(X.T)
Xdf["Y"]=Y

#validation
Y_val=fam2["trait"].astype("int")
Y_val=Y_val-1
Xdf_val=pd.DataFrame(X_val.T)
Xdf_val["Y"]=Y_val

#test
Y_test=fam3["trait"].astype("int")
Y_test=Y_test-1
Xdf_test=pd.DataFrame(X_test.T)
Xdf_test["Y"]=Y_test

''' Getting np arrays '''
x_train=Xdf.drop(['Y'],axis=1).values
y_train=Xdf[['Y']].values

x_val=Xdf_val.drop(['Y'],axis=1).values
y_val=Xdf_val[['Y']].values

x_test=Xdf_test.drop(['Y'],axis=1).values
y_test=Xdf_test[['Y']].values

In [ ]:
''' Function that plot the ROC curve'''
def plot_roc_curve(fpr, tpr, model_name,AUC):
    plt.plot(fpr, tpr, color='orange', label='ROC, AUC='+str(np.round(AUC*100,1))+"%")
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(str(model_name)+' Receiver Operating Characteristic (ROC) Curve on Testing Set')
    
    plt.legend()
    plt.show()

In [ ]:
'''Function that output the validation scores'''
def print_result(model,x_val,y_val,name):
    prediction=model.predict_proba(x_val,y_val)
    print("Validation:")
    print("Accuracy:",accuracy_score(y_val,np.round(prediction[:,1])))
    print("AUC:",roc_auc_score(y_val,prediction[:,1]))
    print("Log Loss:",log_loss(y_val,prediction[:,1]))
    print(classification_report(y_val,np.round(prediction[:,1])))
    return roc_auc_score(y_val,prediction[:,1])

In [ ]:
'''Function that output the test scores and record the information'''
def print_result_test(model,x_test,y_test,name):
    prediction=model.predict_proba(x_test,y_test)
    print("Testing:")
    print("Accuracy:")
    print("AUC:",roc_auc_score(y_val,prediction[:,1]))
    print("Log Loss:",log_loss(y_val,prediction[:,1]))
    print(classification_report(y_val,np.round(prediction[:,1])))
    fpr, tpr, threshold = roc_curve(y_test,prediction[:,1])
    plot_roc_curve(fpr,tpr,name,)
    pd.DataFrame(np.array([fpr,tpr,threshold]).T).to_csv(path_logs+"roc_"+str(name)+"001.csv")

In [ ]:
'''Decision Tree'''

In [ ]:
''' Training model'''
clf2 = DecisionTreeClassifier(random_state=0).fit(x_train, y_train[:,0])

In [ ]:
'''Printing result validation set'''
print_result(clf2,x_val,y_val,"DecisionTree")

In [ ]:
'''Printing result test set'''
print_result_test(clf2,x_test,y_test,"DecisionTree")

In [ ]:
'''Random Forest'''

In [ ]:
''' Training model'''
estimators=[x*500 for x in range(1,10)]
models=[]
for number_estimator in estimators:
    clf3=RandomForestClassifier(n_estimators=number_estimator,random_state=0,n_jobs=40).fit(x_train, y_train[:,0])
    models.append(clf3)

In [ ]:
'''Printing result validation set for each hyper parameter'''
i=0
best_auc=0
local_auc=None
best_model=None
for model in models: 
    local_auc=print_result(model,x_val,y_val,"RandomForest with estimators="+estimators[i])
    if local_auc>best_auc:
        best_auc=local_auc
        best_model=model
    i+=1

In [ ]:
'''Printing result test set'''
print_result_test(best_model,x_test,y_test,"Random Forest")

In [ ]:
'''SVM Classifier'''

In [ ]:
''' Training model'''
Cs=[0.001,0.01,0.1,1,5,10,15,20]
models=[]
for C in Cs:
    clf4 = SVC(C=C).fit(x_train, y_train[:,0])
    models.append(clf4)

In [ ]:
'''Printing result validation set for each hyper parameter'''
i=0
best_auc=0
local_auc=None
best_model=None
for model in models: 
    local_auc=print_result(model,x_val,y_val,"SVM classifier with C="+Cs[i])
    if local_auc>best_auc:
        best_auc=local_auc
        best_model=model
    i+=1

In [ ]:
'''Printing result test set'''
print_result_test(best_model,x_test,y_test,"SVM Classifier")

In [ ]:
'''Naive Bayes'''

In [ ]:
''' Training model'''
clf5=GaussianNB().fit(x_train, y_train[:,0])

In [ ]:
'''Printing result validation set'''
print_result(clf5,x_val,y_val,"Naive Gauss")

In [ ]:
'''Printing result test set'''
print_result_test(clf5,x_test,y_test,"Naive Gauss")

In [ ]:
''' AdaBoost'''

In [ ]:
''' Training model'''
estimators=[x*500 for x in range(1,10)]
models=[]
for number_estimator in estimators:
    clf6=AdaBoostClassifier(n_estimators=number_estimator).fit(x_train, y_train[:,0])
    models.append(clf6)


In [ ]:
'''Printing result validation set for each hyper parameter'''
i=0
best_auc=0
local_auc=None
best_model=None
for model in models: 
    local_auc=print_result(model,x_val,y_val,"AdaBoost with estimators="+estimators[i])
    if local_auc>best_auc:
        best_auc=local_auc
        best_model=model
    i+=1

In [ ]:
'''Printing result test set'''
print_result_test(best_model,x_test,y_test,"AdaBoost")

In [ ]:
''' Gradient Boosting'''

In [ ]:
''' Training model'''
estimators=[x*100 for x in range(1,10)]
models=[]
for number_estimator in estimators:
    clf7=GradientBoostingClassifier(n_estimators=number_estimator).fit(x_train, y_train[:,0])
    models.append(clf7)


In [ ]:
'''Printing result validation set for each hyper parameter'''
i=0
best_auc=0
local_auc=None
best_model=None
for model in models: 
    local_auc=print_result(model,x_val,y_val,"Gradient Boosting with estimators="+estimators[i])
    if local_auc>best_auc:
        best_auc=local_auc
        best_model=model
    i+=1
print("Best Parameter="+best_model.)

In [ ]:
'''Printing result test set'''
print_result_test(best_model,x_test,y_test,"Gradient Boosting")